In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON .. && make -j`nproc`
!mkdir openpose/images openpose/videos 

Selecting previously unselected package libgflags2.2.
(Reading database ... 155219 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

In [ ]:
import sys
OpenposeDir = '/content/openpose/'
pyopenpose_dir = os.path.join(OpenposeDir,'build','python') # ex: '/content/openpose/build/python'
if pyopenpose_dir not in sys.path:
    sys.path.append(pyopenpose_dir)
from openpose import pyopenpose as op

In [ ]:
!pwd

/content


In [ ]:
params = dict()
params['model_folder'] = 'openpose/models/'
params['net_resolution'] = '176x176'
params['number_people_max'] = 1
params['display'] = 0
params['disable_multi_thread'] = True
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

In [ ]:
import sys
import cv2
import os
import argparse
import numpy as np

sys.path.append('/usr/local/python')
from google.colab.patches import cv2_imshow
from openpose import pyopenpose as op
from google.colab.patches import cv2_imshow
params = dict()
params['model_folder'] = 'openpose/models/'
params['net_resolution'] = '176x176'
params['number_people_max'] = 1
params['display'] = 0
params['disable_multi_thread'] = True
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

# Make output dir
if not os.path.exists('/content/drive/MyDrive/LG/openpose_cord'):
    os.makedirs('/content/drive/MyDrive/LG/openpose_cord')

# Read filepaths for images
classes = {}
for f in os.scandir('/content/drive/MyDrive/LG/fine'):
    if f.is_dir():
        filenames = [g.path for g in os.scandir(f.path) if g.is_file()]
        classes[f.name] = filenames

# Start openpose
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

i = 0
# Get coordinates and scores for all images
for c in classes:
    i += 1

    # Prepare file name
    out_dir_name = os.path.join('/content/drive/MyDrive/LG/openpose_cord', c)
    out_arr = []

    for f in classes[c]:
        print(f)
        # Label image
        #print("Labeling image {}".format(f))
        datum = op.Datum()
        imageToProcess = cv2.imread(f)
        #cv2_imshow(imageToProcess)
        if imageToProcess is None:
            continue
        datum.cvInputData = imageToProcess
        opWrapper.emplaceAndPop(op.VectorDatum([datum]))
        #cv2_imshow(datum.cvOutputData)
        if type(datum.poseKeypoints) == np.ndarray and datum.poseKeypoints.size > 0:
            #print(datum.poseKeypoints)
            out_arr.append(datum.poseKeypoints)
            #cv2.imshow("", datum.cvOutputData)
            cv2.waitKey(0)

    # Save file
    np.save(out_dir_name, out_arr)
    print("Done with {}/{}".format(i, len(c)))

print("Complete!")

/content/drive/MyDrive/LG/fine/Triangle/00000021.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000014.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000027.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000020.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000011.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000030.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000003.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000042.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000005.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000007.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000029.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000032.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000024.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000043.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000036.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000009.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000046.jpeg
/content/drive/MyDrive/LG/fine/Triangle/00000026.jpeg
/content/drive/MyDrive/LG/fi

In [ ]:
print(classes.keys())

dict_keys(['Triangle', 'warrior2', 'downdog', 'tree1', 'tree2', 'tree4', 'tree3', 'tree5', 'plank2', 'plank1', 'plank4', 'plank3', 'goddess2', 'goddess1', 'cobra2', 'cobra1'])


In [ ]:
f = np.load('/content/drive/MyDrive/LG/openpose_cord/downdog.npy')
f.shape

(222, 1, 25, 3)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization, Dropout
from keras.callbacks import CSVLogger

In [ ]:
X_data = []
y_data = []
k = 0
for f in os.scandir("/content/drive/MyDrive/LG/openpose_cord"):
  if f.is_file() and f.name != '.DS_Store':
    print(f)
    x = np.load(f)
    #print(x)
    # Remove empty coords
    x = [coords for coords in x if 1 in coords.shape]
    x = np.concatenate(x)
    X_data.append(x)
    y_data.extend([k]*x.shape[0])
    k += 1

X_data = np.concatenate(X_data)
y_data = np.array(y_data)

# Sanity check
print("X_data shape: {}".format(X_data.shape))
print("y_data shape: {}".format(y_data.shape))


<DirEntry 'Triangle.npy'>
<DirEntry 'warrior2.npy'>
<DirEntry 'downdog.npy'>
<DirEntry 'tree1.npy'>
<DirEntry 'tree2.npy'>
<DirEntry 'tree4.npy'>
<DirEntry 'tree3.npy'>
<DirEntry 'tree5.npy'>
<DirEntry 'plank2.npy'>
<DirEntry 'plank1.npy'>
<DirEntry 'plank4.npy'>
<DirEntry 'plank3.npy'>
<DirEntry 'goddess2.npy'>
<DirEntry 'goddess1.npy'>
<DirEntry 'cobra2.npy'>
<DirEntry 'cobra1.npy'>
X_data shape: (1689, 25, 3)
y_data shape: (1689,)


In [ ]:
def scale_transform_normalize(coords):
    """
    Parameters:
    coords (ndarray): array of (x,y,c) coordinates

    Returns:
    ndarray: coords scaled to 1x1 with center at (0,0)
    ndarray: confidence scores of each joint
    """
    coords, scores = coords[:,:,:-1], coords[:,:,-1]
    diff = coords.max(axis=1) - coords.min(axis=1)
    diff_max = np.max(diff, axis=0)
    mean = coords.mean(axis=1).reshape(coords.shape[0],1,coords.shape[-1])
    out = (coords - mean) / diff_max
    
    return out, scores

In [ ]:
N,D,C = X_data.shape

# Prepare X
X_norm, scores = scale_transform_normalize(X_data)
scores = scores.reshape((N, D, 1))
X_data = np.concatenate([X_norm, scores], axis=2)
X_data /= np.linalg.norm(X_data, axis=2)[:, :, np.newaxis]
X = []

# Prepare y
y = []

# Grab every possible combination of 2 rows
for index in tqdm(combinations(np.arange(N), 2)):
    vec_1 = X_data[index[0]]
    vec_2 = X_data[index[1]]
    X.append(np.concatenate([vec_1, vec_2]).flatten())
    y.append(int(y_data[index[0]] == y_data[index[1]]))
    

1425516it [00:08, 159240.20it/s]


In [ ]:
X = np.array(X)
y = np.array(y)
# trues = X[y == 1]
# falses = X[y == 0]
# small_falses = resample(falses, n_samples=trues.shape[0])

# balanced_X = np.concatenate([trues, small_falses])
# balanced_y = np.concatenate([np.ones((trues.shape[0],)), np.zeros((trues.shape[0]))])

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Define network
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Activation('relu'),
    Dense(128),
    BatchNormalization(),
    Activation('relu'),
    Dense(1),
    BatchNormalization(),
    Activation('sigmoid'),
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
X_train.shape

(1282964, 150)

In [ ]:
import collections, numpy
collections.Counter(y_train)

Counter({0: 1129250, 1: 153714})

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/LG/train_log_2'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,verbose=2,
    save_best_only=True)
csv_logger = CSVLogger("model_history_log2.csv", append=True)
history = model.fit(x=X_train, y=y_train,
                    batch_size=32, epochs=100,
                    verbose=2, validation_split=0.1,
                    callbacks=[csv_logger,checkpoint_callback])

Epoch 1/100

Epoch 00001: val_loss improved from inf to 0.10934, saving model to /content/drive/MyDrive/LG/train_log_2/ckpt_1
36084/36084 - 191s - loss: 0.1666 - accuracy: 0.9388 - val_loss: 0.1093 - val_accuracy: 0.9623 - 191s/epoch - 5ms/step
Epoch 2/100

Epoch 00002: val_loss improved from 0.10934 to 0.08086, saving model to /content/drive/MyDrive/LG/train_log_2/ckpt_2
36084/36084 - 190s - loss: 0.1036 - accuracy: 0.9606 - val_loss: 0.0809 - val_accuracy: 0.9689 - 190s/epoch - 5ms/step
Epoch 3/100

Epoch 00003: val_loss improved from 0.08086 to 0.06200, saving model to /content/drive/MyDrive/LG/train_log_2/ckpt_3
36084/36084 - 187s - loss: 0.0877 - accuracy: 0.9669 - val_loss: 0.0620 - val_accuracy: 0.9770 - 187s/epoch - 5ms/step
Epoch 4/100

Epoch 00004: val_loss improved from 0.06200 to 0.06071, saving model to /content/drive/MyDrive/LG/train_log_2/ckpt_4
36084/36084 - 187s - loss: 0.0772 - accuracy: 0.9709 - val_loss: 0.0607 - val_accuracy: 0.9775 - 187s/epoch - 5ms/step
Epoch 5/

KeyboardInterrupt: ignored

In [ ]:
model.predict